# Объединение, очистка датасетов

In [2]:
import pandas as pd

df_V = pd.read_csv('voice_commands.csv')
print(df_V.shape)
df_J = pd.read_csv('voice_commands_1.csv')
print(df_V.shape)
df_D = pd.read_csv('voice_dataset_commands.csv')
print(df_V.shape)
df_S = pd.read_csv('voise_commands.csv')
print(df_V.shape)

(100, 2)
(100, 2)
(100, 2)
(100, 2)


In [3]:
df_S.head()

,text,label
0,Тревога! Занять боевые посты!,Управление оружием
1,Привести орудия в готовность!,Управление оружием
2,"Враг в поле зрения, открыть огонь!",Управление оружием
3,"Атака с левого борта, отражать!",Управление оружием
4,"Эти твари лезут на борт, огонь на поражение!",Управление оружием


In [4]:
lst = [df_V, df_J, df_D, df_S]
df = pd.concat((f for f in lst), ignore_index=True)

df.to_csv("all_commands.csv", index=False)

In [5]:
print(df.shape)
df.head()

(400, 2)


,text,label
0,"Кайл, открывай огонь по врагам!",сражение
1,"Все в бой, не отступать!",сражение
2,"Офицер безопасности, действуй!",сражение
3,"Маркус, прикрой капитана!",сражение
4,Нападайте на захватчиков!,сражение


In [6]:
df['label'].unique()

array(['сражение', 'хил', 'следование', 'пожар', 'Подождите',
       'Устранить утечки', 'Очистить элементы',
       'Ремонт повреждённых систем', 'Ремонт механических систем',
       'Ремонт электрических систем', 'Включение питания реактора',
       'Выключение питания реактора', 'Управление оружием',
       'Навигация назад', 'Навигация к пункту назначения', 'Отстранение'],
      dtype=object)

In [7]:
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.strip()             # убираем пробелы по краям
    text = text.replace('\n', ' ')  # заменяем переносы строк
    return text

df['text'] = df['text'].apply(clean_text)

df.head()


,text,label
0,"Кайл, открывай огонь по врагам!",сражение
1,"Все в бой, не отступать!",сражение
2,"Офицер безопасности, действуй!",сражение
3,"Маркус, прикрой капитана!",сражение
4,Нападайте на захватчиков!,сражение


In [8]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['label_id'] = le.fit_transform(df['label'])

print(le.classes_)
df['label_id'].unique()

['Включение питания реактора' 'Выключение питания реактора'
 'Навигация к пункту назначения' 'Навигация назад' 'Отстранение'
 'Очистить элементы' 'Подождите' 'Ремонт механических систем'
 'Ремонт повреждённых систем' 'Ремонт электрических систем'
 'Управление оружием' 'Устранить утечки' 'пожар' 'следование' 'сражение'
 'хил']


array([14, 15, 13, 12,  6, 11,  5,  8,  7,  9,  0,  1, 10,  3,  2,  4])

# Токенизация

In [9]:
from sklearn.model_selection import train_test_split

train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['text'].tolist(),
    df['label_id'].tolist(),
    test_size=0.2,
    random_state=42,
    stratify=df['label_id']  # использую для равномерного распределения классов
)


In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [11]:
# Пример: токенизация одной фразы
example = "Инженер, туши пожар!"
tokens = tokenizer(example, padding=True, truncation=True, return_tensors="pt")
print(tokens)


{'input_ids': tensor([[    2,  8905, 26456,    16, 11997,  2666,   705, 19062,     5,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [12]:
 # truncation-обрезка больших предложений, padding-дополнение мальеньких предложений, max_length-макс. длина предложения
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=64)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=64)

# Подгрузка и дообучение модели

In [13]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import pandas as pd

class TextDataset(Dataset):
  """
  Класс для dataloader. Помогает искать данные по индексам
  """
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
    # выражение key: torch.tensor(val[idx]) создает словарь
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)

In [39]:
train_dataset = TextDataset(train_encodings, train_labels)
test_dataset = TextDataset(test_encodings, test_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

In [40]:
model = AutoModelForSequenceClassification.from_pretrained("cointegrated/rubert-tiny", num_labels=len(set(df['label'])))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
device

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


device(type='cuda')

In [41]:
optimizer = AdamW(model.parameters(), lr=2e-5)

num_epochs = 40
num_training_steps = num_epochs * len(train_loader)

# lr_scheduler нужен для изменения lr
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [42]:
model.train()
for epoch in range(num_epochs):
  # tqdm для красивого отображения обучения
  loop = tqdm(train_loader, desc=f"Epoch {epoch+1}")
  for batch in loop:
    # выражение k: v.to(device) создает словарь
    batch = {k: v.to(device) for k, v in batch.items()}

    outputs = model(**batch)
    loss = outputs.loss

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    lr_scheduler.step()

    loop.set_postfix(loss=loss.item())


Epoch 40: 100%|██████████| 20/20 [00:00<00:00, 64.46it/s, loss=1.23]


In [43]:
import torch
from sklearn.metrics import accuracy_score, f1_score, classification_report
from tqdm import tqdm

def evaluate(model, data_loader, device):
  model.eval()
  preds, labels = [], []

  with torch.no_grad():
    loop = tqdm(data_loader, desc="Evaluating")
    for batch in loop:
      batch = {k: v.to(device) for k, v in batch.items()}

      outputs = model(**batch)
      logits = outputs.logits
      predictions = torch.argmax(logits, dim=-1)

      preds.extend(predictions.cpu().numpy())
      labels.extend(batch["labels"].cpu().numpy())

  # Метрики
  acc = accuracy_score(labels, preds)
  f1 = f1_score(labels, preds, average="weighted")
  report = classification_report(labels, preds, digits=4)

  print(f"\nAccuracy: {acc:.4f}")
  print(f"F1-score (weighted): {f1:.4f}")
  print("\nDetailed report:\n", report)

  return acc, f1


In [44]:
# Использую данные из обучения (train) для проверки переобучения
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
acc, f1 = evaluate(model, train_loader, device)

Evaluating: 100%|██████████| 20/20 [00:00<00:00, 272.81it/s]


Accuracy: 0.8781
F1-score (weighted): 0.8593

Detailed report:
               precision    recall  f1-score   support

           0     1.0000    0.0500    0.0952        20
           1     0.5128    1.0000    0.6780        20
           2     0.9565    1.0000    0.9778        22
           3     1.0000    0.9500    0.9744        20
           4     1.0000    1.0000    1.0000        23
           5     1.0000    1.0000    1.0000        20
           6     0.9524    1.0000    0.9756        20
           7     1.0000    0.9500    0.9744        20
           8     1.0000    0.9500    0.9744        20
           9     0.9091    1.0000    0.9524        20
          10     1.0000    0.7333    0.8462        15
          11     1.0000    1.0000    1.0000        20
          12     0.9091    1.0000    0.9524        20
          13     0.6667    1.0000    0.8000        20
          14     0.7857    0.5500    0.6471        20
          15     0.9412    0.8000    0.8649        20

    accuracy   

In [45]:
# Тут данные тестовые (test)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
acc, f1 = evaluate(model, test_loader, device)

Evaluating: 100%|██████████| 5/5 [00:00<00:00, 256.39it/s]


Accuracy: 0.7375
F1-score (weighted): 0.7057

Detailed report:
               precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000         5
           1     0.4444    0.8000    0.5714         5
           2     1.0000    1.0000    1.0000         5
           3     0.6250    1.0000    0.7692         5
           4     1.0000    0.8333    0.9091         6
           5     0.8333    1.0000    0.9091         5
           6     0.7143    1.0000    0.8333         5
           7     1.0000    1.0000    1.0000         5
           8     1.0000    1.0000    1.0000         5
           9     1.0000    1.0000    1.0000         5
          10     1.0000    0.5000    0.6667         4
          11     1.0000    0.6000    0.7500         5
          12     0.6667    0.4000    0.5000         5
          13     0.5000    1.0000    0.6667         5
          14     0.7500    0.6000    0.6667         5
          15     0.0000    0.0000    0.0000         5

    accuracy   

In [20]:
model.save_pretrained("rubert_tiny_finetuned")
tokenizer.save_pretrained("rubert_tiny_finetuned")

print("Модель есть, можно и поесть.")

Модель есть, можно и поесть.


# Проверка модели

In [66]:
# модель не понимает 0 и 15 классы вообще! (0 - Включение питания реактора, 15 - хил)
df[df['label_id'] == 0].head()

,text,label,label_id
250,Включи реактор срочно!,Включение питания реактора,0
251,"Инженер, запускай реактор, блин!",Включение питания реактора,0
252,"Питание на реактор, быстро!",Включение питания реактора,0
253,"Реактор в оффлайн, включи!",Включение питания реактора,0
254,Стартуй ядерку немедленно!,Включение питания реактора,0


In [67]:
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")

def classify_text(model, text, device, label_map=None):
    model.eval()
    with torch.no_grad():
        # Токенизация (так же, как при обучении)
        inputs = tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=64
        ).to(device)

        outputs = model(**inputs)
        logits = outputs.logits
        pred_id = torch.argmax(logits, dim=-1).item()

    if label_map:
        return label_map[pred_id]
    else:
        return pred_id


In [68]:
label_map = {
    0: "Включение питания реактора",
    1: "Выключение питания реактора",
    2: "Навигация к пункту назначения",
    3: "Навигация назад",
    4: "Отстранение",
    5: "Очистить элементы",
    6: "Подождите",
    7: "Ремонт механических систем",
    8: "Ремонт повреждённых систем",
    9: "Ремонт электрических систем",
    10: "Управление оружием",
    11: "Устранить утечки",
    12: "пожар",
    13: "следование",
    14: "сражение",
    15: "хил"
}


In [77]:
text = "за мной."
pred = classify_text(model, text, device, label_map)
print("Предсказанный класс:", pred)

Предсказанный класс: следование
